In [1]:
import keras

In [2]:
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='n2Pei4MFrcJQPFfOS1IL8yLOWI0mEi3QVW5OZ0_mCIQK',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfiredetection-donotdelete-pr-jv7mdr7aoigni3'
object_key = 'archive.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [13]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/Dataset/train_set')

In [14]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)

In [15]:
x_train=train_datagen.flow_from_directory('/home/wsuser/work/Dataset/Dataset/train_set',target_size=(64,64),batch_size=32,class_mode='binary')

Found 436 images belonging to 2 classes.


# Applying ImageDataGenerator functionality to test dataset

In [20]:
x_test=test_datagen.flow_from_directory('/home/wsuser/work/Dataset/Dataset/test_set',target_size=(64,64),batch_size=32,class_mode='binary')

Found 118 images belonging to 2 classes.


# Importing Model Building Libraries

In [21]:
#to define the linear Initialisation import sequential
from keras.models import Sequential
#to add layers import Dense
from keras.layers import Dense
#to create Convolutional kernel import convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer 
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

# Initializing the model

In [23]:
model = Sequential()

# Adding CNN Layers

In [24]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
#add maxpooling layers
model.add(MaxPooling2D(pool_size=(2,2)))
#add faltten layer
model.add(Flatten())


# Add Dense layers

In [25]:
#add hidden layers
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

# configuring the learning process

In [26]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

# Training the model

In [27]:
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 17s 1s/step - loss: 0.5463 - accuracy: 0.7500 - val_loss: 0.2049 - val_accuracy: 0.9322
Epoch 2/10
14/14 [==============================] - 16s 1s/step - loss: 0.3025 - accuracy: 0.8761 - val_loss: 0.1367 - val_accuracy: 0.9492
Epoch 3/10
14/14 [==============================] - 16s 1s/step - loss: 0.2297 - accuracy: 0.8991 - val_loss: 0.0949 - val_accuracy: 0.9831
Epoch 4/10
14/14 [==============================] - 16s 1s/step - loss: 0.1966 - accuracy: 0.9128 - val_loss: 0.0957 - val_accuracy: 0.9661
Epoch 5/10
14/14 [==============================] - 16s 1s/step - loss: 0.1876 - accuracy: 0.9197 - val_loss: 0.1235 - val_accuracy: 0.9492
Epoch 6/10
14/14 [==============================] - 15s 1s/step - loss: 0.1727 - accuracy: 0.9197 - val_loss: 0.0858 - val_accuracy: 0.9746
Epoch 7/10
14/14 [==============================] - 16s 1s/step - loss: 0.1677 - accuracy: 0.9312 - val_loss: 0.1725 - val_accuracy: 0.9153
Epoch 8/10
14/14 [==

# Save the model

In [28]:
model.save("forest_fire.h5")

In [30]:
!tar -zcvf image-classification_new.tgz forest_fire.h5

forest_fire.h5


In [31]:
ls  -1

Dataset/
forest1.h5
forest_fire.h5
image-classification_new.tgz


In [32]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 22.4 MB/s eta 0:00:01


In [33]:
from ibm_watson_machine_learning import APIClient
wml_credentilas = {
                   "url":"https://us-south.ml.cloud.ibm.com",
                   "apikey":"3_LDreq5XuQ1mvXmnkNMIaHxQvRog5Lqt8OEyN6hn3NM"
}
client = APIClient(wml_credentilas)

In [34]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [38]:
space_uid = guid_from_space_name(client, 'Deployment_1')
print("Space UID ="+ space_uid)


Space UID =e7da7763-f413-410c-bfc2-303a8c0e6ab2


In [39]:
client.set.default_space(space_uid)

'SUCCESS'

In [40]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

# Predictions

In [41]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [42]:
model_details = client.repository.store_model(model='image-classification_new.tgz',meta_props={ 
client.repository.ModelMetaNames.NAME:'CNN',
client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [43]:
model_id

'9cadfd94-206f-4a85-b36e-0ccaf08133ff'

In [44]:
#import load model from keras.model
from keras.models import load_model
#import image from keras
from tensorflow.keras.preprocessing import image
import numpy as np
#import cv2
import cv2
#load the saved model
model=load_model("forest_fire.h5")
img=image.load_img('/home/wsuser/work/Dataset/Dataset/test_set/with fire/forest_fire_2268729_1280.jpg')
x=image.img_to_array(img)
res=cv2.resize(x,dsize=(64,64),interpolation=cv2.INTER_CUBIC)
#expand the image shape
x=np.expand_dims(res,axis=0)

In [45]:
pred=model.predict(x)
pred = int(pred[0][0])
pred
int(pred)

1

In [46]:
if pred==1:
  print('Forest fire')
elif pred==0:
  print('No Fire')


Forest fire


# Open cv for video processing

In [47]:
pip install twilio

     |████████████████████████████████| 1.4 MB 20.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install playsound

  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7037 sha256=5245566c26d7ffce3bb1d7f736960ea625b8052548f4d4a96f58722aab683bdf
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ba/39/54/c8f7ff9a88a644d3c58b4dec802d90b79a2e0fb2a6b884bf82
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.


In [49]:
from logging import WARNING
#import opencv library
import cv2
#import numpy
import numpy as np
#import image function from keras
from keras.preprocessing import image
#import load_model from keras
from keras.models import load_model
#import client from twilio API
from twilio.rest import Client
#import playsound package
from playsound import playsound

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


# Creating An Account in Twilio Service

In [50]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='n2Pei4MFrcJQPFfOS1IL8yLOWI0mEi3QVW5OZ0_mCIQK',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfiredetection-donotdelete-pr-jv7mdr7aoigni3'
object_key = 'Project Development Phase_Sprint 4_Project Development_Sprint 4_beep.mp3'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [53]:
from twilio.rest import Client
from playsound import playsound
if pred==1:
  print('Forest fire')
  account_sid='AC80e370618f6d64a916a254397e6e962a'
  auth_token='e88ac518336e63eca9a13461f0afbf9e'
  client=Client(account_sid,auth_token)
    
  message=client.messages \
  .create(
      body='forest fire is detected,stay alert',
      #use twilio free number
      from_='+14154087674',
      #to number
      to='+917539964343')
  print(message.sid)
  print("Fire detected")
  print("SMS Sent!")

Forest fire


TwilioRestException: HTTP 400 error: Unable to create record: The From phone number +19803934024 is not a valid, SMS-capable inbound phone number or short code for your account.